<a href="https://colab.research.google.com/github/therobinkay/gender_bias_2021/blob/main/Journal_Analysis_2_(GitHub).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import all necessary packages

import io
import pandas as pd
import os
import itertools
import numpy as np
import scipy.stats
import time

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

import math

from collections import defaultdict
from itertools import combinations as comb

import networkx as nx
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cd "drive/My Drive"

In [4]:
# read all necessary .csv files

data = pd.read_csv("drive/My Drive/codes/data.csv")
dp = pd.read_csv("drive/My Drive/codes/doipacs.csv")
cen = pd.read_csv('drive/My Drive/codes/cen.csv')

cdata = pd.read_csv('drive/My Drive/codes/cdata.csv')

cb = pd.read_csv("drive/My Drive/codes/citationBara.csv")
# extra modification for future analyses
cb['pair'] = "(" + cb["citing_doi"] + ", " + cb["cited_doi"] + ")"
cb['pair2'] = "(" + cb["cited_doi"] + ", " + cb["citing_doi"] + ")"

rhom = pd.read_csv("drive/My Drive/codes/rhom.csv")

## Pairwise Count

In [ ]:
# Create a DataFrame: citing_doi, citing_year, cited_doi

## NOTE THAT: citing_year is not used in this paper but is extracted anyway
##            for convenience, in case future projects want to apply restriction
##            on publication year

dc = data[["doi", "year"]].drop_duplicates(subset = 'doi')
dc = dc.merge(cb, left_on = 'doi', right_on = 'citing_doi', how = 'inner')
dc = dc.drop(['doi'], axis=1)
dc = dc.rename({'year': 'citing_year'}, axis=1)

cols = ['citing_doi', 'citing_year', 'cited_doi']
dc = dc.reindex(columns=cols).sort_values('cited_doi')

In [ ]:
# Observe pairs with different primary author gender

hom_mf = homn[homn["gender1"] != homn["gender2"]].reset_index(drop=True)

test = hom_mf.merge(dc, left_on = 'paper1', right_on = 'cited_doi', how = 'left')
test = test.drop(["cited_doi", "citing_year"], axis=1).drop_duplicates(subset=['paper1', 'citing_doi'])
test['count'] = test.groupby(['paper1'])['paper1'].transform('count')
test = test.drop(["citing_doi"], axis=1)
test = test.rename({'count': 'count1'}, axis=1)

test = test.drop_duplicates(subset=['paper1', 'paper2']).drop(['gender1', 'year1', 'paper2', 'gender2', 'year2', 'qval', 'k'], axis=1)
hom_mf = hom_mf.merge(test, on='paper1', how='inner')

test = hom_mf.merge(dc, left_on = 'paper2', right_on = 'cited_doi', how = 'left')
test = test.drop(["cited_doi", "citing_year"], axis=1).drop_duplicates(subset=['paper2', 'citing_doi'])
test['count'] = test.groupby(['paper2'])['paper2'].transform('count')
test = test.drop(["citing_doi"], axis=1)
test = test.rename({'count': 'count2'}, axis=1)

test = test.drop_duplicates(subset=['paper1', 'paper2']).drop(['gender1', 'year1', 'paper1', 'gender2', 'count1', 'year2', 'qval', 'k'], axis=1)
hom_mf = hom_mf.merge(test, on='paper2', how='inner')

In [ ]:
# Calculate c_d (male citation count - female citation count)

listd = []

for c in range(len(hom_mf)):
  if hom_mf['gender1'][c] == 'male':
    listd.append(hom_mf['count1'][c] - hom_mf['count2'][c])
  else:
    listd.append(hom_mf['count2'][c] - hom_mf['count1'][c])

hom_mf['countd'] = listd

In [ ]:
# Extract subDataFrame with q-value less than p threshold

p = 0.006 # Select p thresholds with listed numbers in the thesis

homp = hom_mf[hom_mf['qval'] < p].sort_values('countd').reset_index(drop=True)

mlist = []
fmlist = []

for i in range(len(homp)):
  if homp['gender1'][i] == 'male':
    mlist.append(homp['count1'][i])
    fmlist.append(homp['count2'][i])
  else:
    mlist.append(homp['count2'][i])
    fmlist.append(homp['count1'][i])

# Find the standard deviation
sd = math.sqrt((np.std(mlist) ** 2 + np.std(fmlist) ** 2)/len(homp))

# Find z-scores (for p-values)
print((np.mean(mlist)-np.mean(fmlist))/sd)

## PageRank for Homophily

### Analysis

In [ ]:
# Calculate PageRank Centrality for all network nodes
Graphtype = nx.DiGraph()
C = nx.from_pandas_edgelist(cen, source='citing_doi', target='cited_doi',
                            create_using=Graphtype)
pr = nx.pagerank(C)

# Create DataFrame with doi, id, gender, and centrality
prs = pd.DataFrame()
prs['doi'] = pr.keys()
prs['centrality'] = pr.values()
# prs.sort_values('centrality', ascending=False).head()

nodedata = data[data['is_alpha'] == False]
nodedata = nodedata[nodedata['order'] == 1]
nodedata = nodedata.drop(['order', 'numAuthor',	'is_last',	'is_alpha',	'year',
                          'articleType',	'journal', 'id'], axis=1)
nodedata = nodedata.reset_index()
nodedata.columns = ['id', 'Label', 'Gender']

prs = prs.merge(nodedata, left_on='doi', right_on='Label', how='left')
prs = prs.drop(['Label'], axis=1)
prs = prs[["doi", 'id', 'Gender', 'centrality']]
prs = prs.sort_values('centrality', ascending=False)

# prs.head()

In [ ]:
# Calculate cen_d (male centrality - female centrality)

ph = pd.merge(homn, prs, left_on = "paper1", right_on = "doi", how = "left")
ph = ph.drop(["doi", "id", "Gender"], axis=1)
ph = ph.rename(columns={"centrality": "cen1"})

ph = ph.merge(prs, left_on = "paper2", right_on = "doi", how = "left")
ph = ph.drop(["doi", "id", "Gender"], axis=1)
ph = ph.rename(columns={"centrality": "cen2"})
print(len(ph))

ph = ph[ph['gender1'] != ph['gender2']].reset_index(drop=True)

listd = []

for c in range(len(ph)):
  if ph['gender1'][c] == 'male':
    listd.append(ph['cen1'][c] - ph['cen2'][c])
  else:
    listd.append(ph['cen2'][c] - ph['cen1'][c])

ph['cen_d'] = listd

print(len(ph))
ph.head()

1189255
184694


,paper1,gender1,year1,paper2,gender2,year2,qval,k,cen1,cen2,cen_d
0,10.1103/PhysRevA.30.336,male,1984,10.1103/PhysRevA.34.4472,female,1986,0.017857,False,0.000018,0.000004,1.402271e-05
1,10.1103/PhysRevA.34.4472,female,1986,10.1103/PhysRevA.41.784,male,1990,0.017857,False,0.000004,0.000005,7.764608e-07
2,10.1103/PhysRevA.36.4950,female,1987,10.1103/PhysRevA.41.784,male,1990,0.006410,False,0.000002,0.000005,2.824713e-06
3,10.1103/PhysRevA.39.5466,female,1989,10.1103/PhysRevA.41.784,male,1990,0.006410,False,0.000002,0.000005,2.166712e-06
4,10.1103/PhysRevA.41.3081,female,1990,10.1103/PhysRevA.41.784,male,1990,0.000511,False,0.000002,0.000005,2.824713e-06


In [ ]:
# Extract subDataFrame with q-value less than p threshold

p = 0.002 # Select p thresholds with listed numbers in the thesis

phsub = ph[ph['qval'] < p].sort_values('cen_d').reset_index(drop=True)

mlist = []
fmlist = []

for i in range(len(phsub)):
  if phsub['gender1'][i] == 'male':
    mlist.append(phsub['cen1'][i])
    fmlist.append(phsub['cen2'][i])
  else:
    mlist.append(phsub['cen2'][i])
    fmlist.append(phsub['cen1'][i])

# Find the standard deviation
sd = math.sqrt((np.std(mlist) ** 2 + np.std(fmlist) ** 2)/len(phsub))

# Find z-scores (for p-values)
print((np.mean(mlist)-np.mean(fmlist))/sd)

-0.6457704432323805
